In [2]:
%load_ext autoreload
%autoreload 2

import pathlib

import pandas as pd
pd.set_option('display.float_format', lambda x: '%.1f' % x)

import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams["font.size"] = "20"

from data_handlers.network import parse_bwms
from data_handlers.procmem import parse_pidstat

In [3]:
experiment_path = pathlib.Path(f"/home/jovyan/work/binary_files")

In [4]:
network_df = parse_bwms(experiment_path)

In [5]:
network_df.reset_index().routing.unique()

array(['binary_spray', 'prophet', 'dtlsr', 'epidemic', 'context_epidemic',
       'context_spray', 'context_complex'], dtype=object)

In [6]:
procmem_df = parse_pidstat(experiment_path)

In [7]:
procmem_df.reset_index().routing.unique()

array(['binary_spray', 'prophet', 'dtlsr', 'epidemic', 'context_epidemic',
       'context_spray', 'context_complex'], dtype=object)

In [8]:
merged = procmem_df.merge(network_df, left_index=True, right_index=True).reset_index()

In [9]:
cleared = merged[(merged['routing'] != "context_complex") & (merged['routing'] != "dtlsr") & (merged['routing'] != "prophet")]
cleared['%CPU'] = (cleared['%CPU'] - cleared['%CPU'].min()) / (cleared['%CPU'].max() - cleared['%CPU'].min())
cleared.loc[cleared['routing'] == 'epidemic', 'routing'] = 'a'
cleared.loc[cleared['routing'] == 'context_epidemic', 'routing'] = 'b'
cleared.loc[cleared['routing'] == 'binary_spray', 'routing'] = 'c'
cleared.loc[cleared['routing'] == 'context_spray', 'routing'] = 'd'

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=2)
plt.subplots_adjust(wspace=.3, hspace=.4)

fig.set_size_inches(15,10)

color_mbit = 'C0'
color_cpu = 'C1'

for ax, (name, routing_df) in zip(list(axs[0]) + list(axs[1]), cleared.groupby(['routing'])):
    print(f"Starting {name}")
    ax.tick_params(axis='y', labelcolor=color_mbit)
    ax.set_xlim(-10, 3700)
    ax.set_ylim(0.01, 320)
    
    if name == "c":
        name = "Binary Spray & Wait"
    if name == "b":
        name = "Software Defined Epidemic"
    if name == "d":
        name = "Software Defined Binary Spray & Wait"
    if name == "a":
        name = "Epidemic"
    ax.set_title(name, fontsize=20)
    
    ax2 = ax.twinx()
    ax2.tick_params(axis='y', labelcolor=color_cpu)
    ax2.set_ylim(.01, .2)
                    
    routing_df = routing_df.reset_index()
    
    #routing_df = routing_df.sample(1000)

    print(f"Plotting bandwidth for {name}")
    sns.lineplot(
        x=routing_df['dt'],
        y=routing_df['Mbit/s'].rolling(100).mean(),
        ax=ax,
        color=color_mbit
    )

    print(f"Plotting CPU for {name}")
    sns.lineplot(
        x=routing_df['dt'],
        y=routing_df['%CPU'].rolling(100).mean(),
        ax=ax2,
        color=color_cpu
    )

    if ax.figbox.x0 == 0.125: #Just some magic number. You can find it with print(ax.figbox)
        ax.set_ylabel('Mbit/s', color=color_mbit)
        ax2.set_ylabel('')
    if ax.figbox.x0 == 0.5630434782608696: #Just some magic number. You can find it with print(ax.figbox)
        ax.set_ylabel('', color=color_mbit)
        ax2.set_ylabel('%CPU', color=color_cpu)

    if ax.figbox.y0 == 0.125: #Just some magic number. You can find it with print(ax.figbox)
        ax.set_xlabel('Time (s)')
    if ax.figbox.y0 == 0.5654166666666667: #Just some magic number. You can find it with print(ax.figbox)
        ax.set_xlabel('')
        
    ttl = ax.title
    ttl.set_position([.5, 1.05])
                        
plt.savefig('cpu_network.pdf')
#plt.show()

Starting a
Plotting bandwidth for Epidemic
Plotting CPU for Epidemic
Starting b
Plotting bandwidth for Software Defined Epidemic
